In [16]:
import sqlite3 as db

import pandas as pd
import numpy as np

from utils.clients import OandaClient
from utils.trade_urls import *

import sys
import json
import datetime 
import Keys
import warnings 
warnings.filterwarnings('ignore')
import oandapyV20 as oanda
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20 import API
from oandapyV20.contrib.requests import MarketOrderRequest as OandaMarketOrder
import oandapyV20.endpoints.orders as orders

from environments.live_environments import OandaTradingEnv
from neuralforecast import NeuralForecast
from ray.rllib.policy.policy import Policy
from utils.utils import build_market_image, sharpe_reward
from utils.mappings import symbol_map, binanace_col_map

import pytorch_lightning

pytorch_lightning._logger.setLevel(0)

In [17]:

pair=target_symbol='USD_JPY'
base_asset=pair.split('_')[0]
quote_asset=pair.split('_')[1]


In [18]:
api_key=Keys.oanda_paper_api_key
account_id=Keys.oanda_paper_account

In [29]:
model=NeuralForecast.load("FX_forecasting_model",verbose=0)


Seed set to 2
Seed set to 2
Seed set to 16
Seed set to 16
Seed set to 8
Seed set to 8
Seed set to 5
Seed set to 5


In [35]:
import torch

'FX_onnx_forecasting_model_AutoPatchTST'

In [47]:
sub_model=model.models[0]
# sub_model.to_onnx(f'FX_onnx_forecasting_model{sub_model.__repr__}')
# sub_model.example_input_array=np.zeros(sub_model.input_size)
input_sample=np.zeros((sub_model.input_size,sub_model.h,1))
# model_name=f'FX_onnx_forecasting_model_{sub_model.__repr__()}'
sub_model.hparams



"activation":                   gelu
"alias":                        None
"attn_dropout":                 0.0
"batch_normalization":          False
"batch_size":                   128
"drop_last_loader":             False
"dropout":                      0.2
"early_stop_patience_steps":    -1
"enable_progress_bar":          False
"encoder_layers":               3
"exclude_insample_y":           False
"fc_dropout":                   0.2
"futr_exog_list":               None
"h":                            6
"head_dropout":                 0.0
"hidden_size":                  128
"hist_exog_list":               None
"inference_windows_batch_size": 1024
"input_size":                   18
"learn_pos_embed":              True
"learning_rate":                0.0007622491439073963
"linear_hidden_size":           256
"loss":                         MSE()
"max_steps":                    5000
"n_heads":                      4
"num_lr_decays":                -1
"num_workers_loader":           0
"opt

In [ ]:
for sub_model in model.models:
    

In [6]:
agent_dir='Agent/final_checkpoints/policies/default_policy'
agent= Policy.from_checkpoint(agent_dir)

In [7]:
live_env_config=dict(
                name=f'{target_symbol}_train',
                model=model,
                agent=agent,
                api_key=api_key,
                account_id=account_id,
                test_net=True,
                restore_trading=False,
                target_symbol=target_symbol,
                time_frame='1h',
                reward_function=sharpe_reward,
                positions = [ -.5,-.25,.25, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0,
                )

In [8]:
mktOrder=OandaMarketOrder(
                instrument=pair,
                units=10,

                
                )
print(mktOrder.data)


order_params = orders.OrderCreate(account_id, data=mktOrder.data)
print(order_params)

{'order': {'type': 'MARKET', 'timeInForce': 'FOK', 'instrument': 'USD_JPY', 'units': '10', 'positionFill': 'DEFAULT'}}
v3/accounts/101-001-22524644-005/orders


In [9]:
from oandapyV20.endpoints.pricing import PricingStream

In [10]:
self = OandaTradingEnv(**live_env_config)


In [11]:
self.client._account

{'guaranteedStopLossOrderMode': 'DISABLED',
 'hedgingEnabled': False,
 'id': '101-001-22524644-005',
 'createdTime': '2024-06-17T18:20:04.037670480Z',
 'currency': 'USD',
 'createdByUserID': 22524644,
 'alias': 'RL_trader617',
 'marginRate': '0.02',
 'lastTransactionID': '153',
 'balance': '9994.5161',
 'openTradeCount': 0,
 'openPositionCount': 0,
 'pendingOrderCount': 0,
 'pl': '-1.0164',
 'resettablePL': '-1.0164',
 'resettablePLTime': '0',
 'financing': '-4.4675',
 'commission': '0.0000',
 'dividendAdjustment': '0',
 'guaranteedExecutionFees': '0.0000',
 'orders': [],
 'unrealizedPL': '0.0000',
 'NAV': '9994.5161',
 'marginUsed': '0.0000',
 'marginAvailable': '9994.5161',
 'positionValue': '0.0000',
 'marginCloseoutUnrealizedPL': '0.0000',
 'marginCloseoutNAV': '9994.5161',
 'marginCloseoutMarginUsed': '0.0000',
 'marginCloseoutPositionValue': '0.0000',
 'marginCloseoutPercent': '0.00000',
 'withdrawalLimit': '9994.5161',
 'marginCallMarginUsed': '0.0000',
 'marginCallPercent': '0.

In [12]:
self.client.get_symbol_info()

{'type': 'PRICE',
 'time': '2024-07-01T19:23:09.513895947Z',
 'bids': [{'price': '161.466', 'liquidity': 500000},
  {'price': '161.465', 'liquidity': 2500000},
  {'price': '161.463', 'liquidity': 2000000},
  {'price': '161.460', 'liquidity': 5000000}],
 'asks': [{'price': '161.481', 'liquidity': 500000},
  {'price': '161.482', 'liquidity': 500000},
  {'price': '161.483', 'liquidity': 2000000},
  {'price': '161.485', 'liquidity': 2000000},
  {'price': '161.487', 'liquidity': 5000000}],
 'closeoutBid': '161.446',
 'closeoutAsk': '161.501',
 'status': 'tradeable',
 'tradeable': True,
 'quoteHomeConversionFactors': {'positiveUnits': '0.00619268',
  'negativeUnits': '0.00619325'},
 'instrument': 'USD_JPY'}

In [13]:
obs,info=self.reset(reset_account=True)
obs

Empty DataFrame
Columns: []
Index: []


array([ 1.6091061e+02,  6.0238528e+00,  1.5643320e+02,  2.4308258e+02,
        1.6101401e+02,  1.6089827e+02,  6.0420933e+00,  1.6100119e+02,
        2.7986514e+02,  1.6100999e+02,  1.6091504e+02,  6.0288467e+00,
        1.6449919e+02,  1.9050620e+02,  1.6106799e+02,  1.6090736e+02,
        5.9636574e+00,  1.4775655e+02,  2.3935020e+02,  1.6112500e+02,
        1.6091252e+02,  5.9402781e+00,  1.6289398e+02,  2.2744930e+02,
        1.6110400e+02,  1.6090013e+02,  5.9422421e+00,  1.5925748e+02,
        2.3909331e+02,  1.6135400e+02,  2.1739131e-02, -5.0000000e-01,
       -5.0000000e-01, -1.3698631e-03,  1.6097495e+02,  1.6084927e+02,
        1.5971036e+02,  1.4202477e-03,  5.3243744e-01,  2.1093914e-03,
       -6.6540889e-02,  2.8176550e-03,  5.5452675e-01,  3.2279319e-03,
        3.2613087e+00,  3.1222384e-03,  1.4500892e-01,  4.3726233e-03,
       -2.7178321e-02,  2.5939385e-03,  1.0212636e-01,  0.0000000e+00,
        0.0000000e+00], dtype=float32)

In [14]:
self._trade(.5)

--------------- TRADE ------------------
Position_change 0.5
portfoliio {'asset': 9994.5161, 'fiat': 0, 'borrowed_asset': 0, 'borrowed_fiat': 0, 'interest_asset': 0, 'interest_fiat': 0, 'valuation': 1613839.5009572501}

              Trade 0->0.5 

              so BUY 4997.26 USD units or 

              806919.75 JPY 
              at 161.4725 JPY per USD
            
{'symbol': 'USD_JPY', 'side': 'BUY', 'type': 'MARKET', 'time_in_force': 'ioc', 'notional': 4997.0}


Streaming_Steps

In [15]:
api=API(access_token=api_key)
asfdsf

NameError: name 'asfdsf' is not defined

In [ ]:
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.exceptions import StreamTerminated

In [ ]:
instruments = ["EUR_USD"]
r = pricing.PricingStream(accountID=account_id, params={"instruments": ",".join(instruments)})

n = 0
stopAfter = 3 # let's terminate after receiving 3 ticks
try:
    # the stream requests returns a generator so we can do ...
    for tick in api.request(r):
        print(tick)
        time_stamp=pd.Timestamp(tick['time'])
        print (f' Minutes {time_stamp.minute}')

        if n >= stopAfter:
            r.terminate()
        n += 1
        
except StreamTerminated as err:
    print("Stream processing ended because we made it stop after {} ticks".format(n))

In [ ]:
self.client.update_positions()
self.client._positions

In [ ]:
req=accounts.AccountDetails(self.account_id)
resp=self.api.request(req)['account']
positions=resp.pop('positions')
trades=resp.pop('trades')
positions


In [ ]:
delta=pd.Timedelta(self.time_frame)*(48*5)
now=datetime.datetime.now()
start_date=(now-delta)
start_date=start_date.strftime('%Y-%m-%dT%H:%M:%SZ')
now=now.strftime('%Y-%m-%dT%H:%M:%SZ')
start_date,now

In [ ]:
self.account()

In [ ]:
self.symbol

In [ ]:
price=self.ticker_price(self.symbol)
price

In [ ]:
trade_info=self.get_trade_rules()
min_asset_size=trade_info['min_asset_size']
min_quote_size=min_asset_size*price
prec=trade_info['quote_asset_precision']

size=min_asset_size*price
quant=round(size,prec)
quant_norm=max([min_asset_size,quant])
quant,quant_norm,min_asset_size

In [ ]:
quant=quant_norm
params = {
    'symbol': self.symbol,
    'side':"buy",
    'type': 'MARKET',
    # 'qty':min_asset_size
    'notional':quant,
    "time_in_force":'ioc'
    }

In [ ]:
# preparing market order
market_order_data = self.order(params=params)
market_order_data

In [ ]:
positions = self.trade_client.get_all_positions()
position_list=[p.dict() for p in positions]
pos_frame=pd.DataFrame.from_dict(position_list)
pos_frame['symbol']
pos_frame=pos_frame.set_index('symbol')
pos_frame['qty_available'].to_dict()

In [ ]:
def get_asset_info(self,pair):
    info=trading_client.get_asset(pair)
    info=info.dict()


In [ ]:
def set_trade_rules(self):
    assets=trading_client.get_all_assets()
    asset_df=pd.DataFrame.from_dict([asset.dict() for asset in assets])
    asset_df=asset_df.set_index('symbol')
    asset_df.loc[pair]

In [ ]:
account=trading_client.get_account()
account.dict()

In [ ]:

cash=account['cash']
int(cash)

In [ ]:
renderer=LiveRenderer('Trade_history/trade.db')

In [ ]:
renderer=LiveRenderer('Trade_history/trade.db')
renderer.run()


In [ ]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

cursor.execute(sql_query)
render_names=[t[0] for t in cursor.fetchall()]
name=render_names[0]
df=pd.read_sql(f'select * from {name}',conn)
df

In [ ]:
# df.columns=[c.replace('data_','') for c in df.columns]
# df['date']=df['date'].apply(pd.to_datetime)
# df=df.set_index('date')
df

In [ ]:
# df=df[~df.close.isna()]
mask=df[(df['date']=='62')==True].index
df.loc[mask,'date']=df.loc[mask,'ds'].copy()


In [ ]:
df

In [ ]:
df.to_sql(render_names[0],conn,if_exists='replace',index=False)

In [ ]:
import datetime

In [ ]:
df.iloc[[-1]].to_dict('records')[0]

In [ ]:
pd.Timestamp(datetime.datetime.now())